In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install -q tensorflow-model-optimization

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 2.9 MB/s eta 0:00:00


In [ ]:
import sys
sys.path.append('/content/drive/MyDrive/deep_learning_quantized')

import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import architectures
import network_training
import preprocessing

In [ ]:
def read_data_from_csv(csv_file_name):
      data = pd.read_csv(csv_file_name)
      x_data = data.iloc[:, :-2].values
      y_data = data.iloc[:, -2].values
      subj = data.iloc[:, -1].values

      return x_data, y_data, subj

In [ ]:
csv_file_name = '/content/drive/MyDrive/deep_learning_quantized/multimodal_data.csv'
labels = ('low', 'medium', 'high')
avg = True
fs = 64
step_size = 8

# csv_file_name = '/content/drive/MyDrive/deep_learning_quantized/combined_data.csv'
# labels = ('rest', 'squat', 'step')
# avg = False
# fs = 400
# step_size = 32

features, targets, subj_data = read_data_from_csv(csv_file_name)

In [ ]:
# features = features[:, 0:4]
features = features[:, 0].reshape(-1, 1)
# features = features[:, 1:4]

print(features)
print(targets)

[[ -18.68]
 [ -17.01]
 [ -14.2 ]
 ...
 [-116.5 ]
 [-116.5 ]
 [-113.31]]
[ 1.64195064e-16 -5.50153869e-03 -1.00857702e-02 ...  1.03032917e-02
  9.13208838e-03  8.07195245e-03]


In [ ]:
params = {
    "seed" : 256,
    "learning_rate" : 0.002,
    "weight_decay" : 1e-6,
    "step_size" : 3,
    "gamma" : 0.8,
    "batch_size" : 128,
    "epochs" : 30,
    "num_resblocks" : 1,
}

params["window_size"] = 256 #fs * 4
params["overlap"] = params["window_size"] * 7//8

In [ ]:
from scipy import fft
import matplotlib.pyplot as plt
import numpy as np

# scaled_data = preprocessing.scale_data(features, targets)
scaled_data = features
# scaled_data = features / np.max(features)
# scaled_data = preprocessing.resample_signal(scaled_data, 400, 20)
scaled_data = scaled_data[::step_size]
targ = targets[::step_size]
print(scaled_data.shape, targ.shape)

sliding_X_data, sliding_y_data = preprocessing.apply_sliding_window(scaled_data, targ, subj_data, params["window_size"], params["overlap"], avg)

X_data = sliding_X_data.astype(np.float32)
y_data = sliding_y_data.astype(np.uint8)

# print(X_data.shape, y_data.shape)
# plt.figure(figsize=(6, 4))
# plt.subplot(2, 1, 1)
# plt.plot(X_data[0, :], color='blue')
# plt.xlabel('n')
# plt.ylabel('x(n)')
# plt.subplot(2, 1, 2)
# plt.stem(np.abs(fft.fft(X_data[0, :], axis=0))[:params["window_size"]//2], 'r')
# plt.xlabel('m')
# plt.ylabel('$|X_m|$')

# plt.savefig('original_signal.eps', format='eps')
plt.show()

print(X_data.shape)

(326120, 1) (326120,)
(10184, 256, 1)


In [ ]:
params["num_channels"] = X_data.shape[2]
params["num_classes"] = len(labels)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models

def create_functional_resnet(input_shape, num_classes, num_resblocks):
    inputs = layers.Input(shape=input_shape)

    # x = layers.MaxPooling2D(pool_size=(3, 1), strides=(2, 1), padding='same')(inputs)
    x = layers.Conv2D(4, kernel_size=(3, 1), strides=(2, 1), padding='same', use_bias=False)(inputs)

    x = layers.Conv2D(16, kernel_size=(5, 1), strides=(4, 1), padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = layers.Conv2D(32, kernel_size=(3, 1), strides=(2, 1), padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    x = layers.Conv2D(64, kernel_size=(3, 1), strides=(2, 1), padding='same', use_bias=False)(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    for _ in range(num_resblocks):
        shortcut = x
        x = layers.Conv2D(64, kernel_size=(1, 1), strides=(1, 1), padding='same', use_bias=False)(x)
        x = layers.BatchNormalization()(x)
        x = layers.ReLU()(x)

        x = layers.DepthwiseConv2D(kernel_size=(3, 1), strides=(1, 1), padding='same', use_bias=False)(x)
        x = layers.BatchNormalization()(x)
        x = layers.ReLU()(x)

        x = layers.Conv2D(64, kernel_size=(1, 1), strides=(1, 1), padding='same', use_bias=False)(x)
        x = layers.BatchNormalization()(x)
        x = layers.ReLU()(x)

        x = layers.Add()([x, shortcut])

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.2)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = models.Model(inputs=inputs, outputs=outputs)

    return model

In [ ]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
from keras.models import Sequential
import keras.optimizers
from keras.metrics import Precision, Recall
from sklearn.metrics import f1_score
import os
import time
from keras.callbacks import LearningRateScheduler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import tensorflow_model_optimization as tfmot

def train_model(params, X_train, y_train, X_test, y_test, pruning = False):
    def lr_schedule(epoch, lr):
      if epoch % params['step_size'] == 0 and epoch != 0:
        return lr * params['gamma']
      return lr

    lr_scheduler = LearningRateScheduler(lr_schedule)
    optimizer = keras.optimizers.AdamW(learning_rate=params['learning_rate'])

    precision = Precision()
    recall = Recall()

    def f1_metric(y_true, y_pred):
        precision_value = precision(y_true, y_pred)
        recall_value = recall(y_true, y_pred)
        return 2 * ((precision_value * recall_value) / (precision_value + recall_value + 1e-7))

    input_shape = X_train.shape[1:]
    model = create_functional_resnet(input_shape, params['num_classes'], params['num_resblocks'])
    print(model.summary())

    model.compile(optimizer=optimizer, loss='categorical_crossentropy',
                  metrics=['accuracy'])

    if pruning:
      model_for_pruning = network_training.get_pruned_model(model, X_data, params)

      callbacks = [
          tfmot.sparsity.keras.UpdatePruningStep(),
          lr_scheduler
        ]

      model_for_pruning.compile(optimizer=optimizer, loss='categorical_crossentropy',
                    metrics=['accuracy'])

      model_for_pruning.fit(X_train, y_train, batch_size=params['batch_size'],
                epochs=params['epochs'], verbose=1, validation_data=(X_test, y_test),
                callbacks=callbacks)

    else:
      model.fit(X_train, y_train, batch_size=params['batch_size'],
              epochs=params['epochs'], verbose=1, validation_data=(X_test, y_test),
              callbacks=[lr_scheduler])
      model_for_pruning = model;

    start_time = time.time()
    y_pred = model.predict(X_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true_classes = np.argmax(y_test, axis=1)
    non_quantized_time = time.time() - start_time

    accuracy = accuracy_score(y_true_classes, y_pred_classes)
    precision = precision_score(y_true_classes, y_pred_classes, average='macro')
    recall = recall_score(y_true_classes, y_pred_classes, average='macro')
    f1 = f1_score(y_true_classes, y_pred_classes, average='macro')

    return model, accuracy, f1, precision, recall, non_quantized_time

In [ ]:
import tensorflow_model_optimization as tfmot
from sklearn.model_selection import KFold

def training_loop(X_data, y_data, params, crossvalid = False):
  if crossvalid:
    num_folds = 5
    kf = KFold(n_splits=num_folds, shuffle=True, random_state = params["seed"])
    data_splits = [(X_data[train_ind], X_data[val_ind], y_data[train_ind], y_data[val_ind]) for train_ind, val_ind in kf.split(X_data)]
  else:
    num_folds = 1
    X_train, X_val, y_train, y_val = network_training.split_data(X_data, y_data, train_size=0.8)
    data_splits = [(X_train, X_val, y_train, y_val)]

  total_accuracy = []
  total_f1_score = []
  total_precision = []
  total_recall = []
  total_non_quantized_time = []

  total_accuracy_quant = []
  total_f1_score_quant = []
  total_precision_quant = []
  total_recall_quant = []
  total_quantized_time = []

  for fold, (X_train_fold, X_val_fold, y_train_fold, y_val_fold) in enumerate(data_splits):
      print(f"Fold {fold+1}/{num_folds}")

      y_train_fold = keras.utils.to_categorical(y_train_fold, num_classes=params['num_classes'])
      y_val_fold = keras.utils.to_categorical(y_val_fold, num_classes=params['num_classes'])

      X_train_fold = np.expand_dims(X_train_fold, axis=-1)
      X_val_fold = np.expand_dims(X_val_fold, axis=-1)

      model, accuracy, f1_score, precision, recall, non_quantized_time = train_model(params, X_train_fold, y_train_fold, X_val_fold, y_val_fold, pruning=False)

      tflite_quant_model = network_training.get_quantized_model(model, X_train_fold)
      interpreter = network_training.get_tflite_interpreter(tflite_quant_model)
      accuracy_quant, f1_score_quant, precision_quant, recall_quant, quantized_time = network_training.evaluate_quantized_metrics(interpreter, X_val_fold, y_val_fold)

      start_time = time.time()
      score = model.evaluate(X_val_fold, y_val_fold, verbose=0)
      non_quantized_time = time.time() - start_time

      print(f"""Non-quantized: acc - {accuracy}, f1 - {f1_score}, prec - {precision}, rec - {recall}""")
      print(f"""Quantized: acc - {accuracy_quant}, f1 - {f1_score_quant}, prec - {precision_quant}, rec - {quantized_time}""")
      print(f"""Non-quantized time: {non_quantized_time}; Quantized time: {quantized_time} """)

      if fold == 0:
        # print(model.summary())
        non_quantized_model_size, quantized_model_size =  network_training.compare_model_sizes(tflite_quant_model, model)
        print(f"""Non-quantized size: {non_quantized_model_size}; Quantized size: {quantized_model_size} KB """)

      total_accuracy.append(accuracy)
      total_f1_score.append(f1_score)
      total_precision.append(precision)
      total_recall.append(recall)
      total_non_quantized_time.append(non_quantized_time)

      total_accuracy_quant.append(accuracy_quant)
      total_f1_score_quant.append(f1_score_quant)
      total_precision_quant.append(precision_quant)
      total_recall_quant.append(recall_quant)
      total_quantized_time.append(quantized_time)

  mean_accuracies = np.mean(total_accuracy)
  mean_f1_scores = np.mean(total_f1_score)
  mean_precisions = np.mean(total_precision)
  mean_recalls = np.mean(total_recall)
  mean_times = np.mean(total_non_quantized_time)

  mean_accuracies_quant = np.mean(total_accuracy_quant)
  mean_f1_scores_quant = np.mean(total_f1_score_quant)
  mean_precisions_quant = np.mean(total_precision_quant)
  mean_recalls_quant = np.mean(total_recall_quant)
  mean_times_quant = np.mean(total_quantized_time)
  print(f"""Non-quantized:  acc - {mean_accuracies*100:.2f} F1 - {mean_f1_scores*100:.2f}
        prec - {mean_precisions*100:.2f} rec - {mean_recalls*100:.2f}
        time [ms] - {mean_times*1000:.2f} """)

  print(f"""Quantized:  acc - {mean_accuracies_quant*100:.2f} F1 - {mean_f1_scores_quant*100:.2f}
        prec - {mean_precisions_quant*100:.2f} rec - {mean_recalls_quant*100:.2f}
        time [ms] - {mean_times_quant*1000:.2f} """)

In [ ]:
import keras.utils

keras.utils.set_random_seed(params["seed"])
np.random.seed(params["seed"])
training_loop(X_data, y_data, params, False)

Fold 1/1
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 1, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 128, 1, 4)         12        
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 1, 16)         320       
                                                                 
 batch_normalization (Batch  (None, 32, 1, 16)         64        
 Normalization)                                                  
                                                                 
 re_lu (ReLU)                (None, 32, 1, 16)         0         
                                                                 
 conv2d_2 (Conv2D)           (None, 16, 1, 32)         1536      
                                                    

In [ ]:
# from sklearn.model_selection import KFold
# import keras.utils
# import time

# total_accuracy = []
# total_f1_score = []
# total_precision = []
# total_recall = []
# total_non_quantized_time = []

# total_accuracy_quant = []
# total_f1_score_quant = []
# total_precision_quant = []
# total_recall_quant = []
# total_quantized_time = []

# def training_loop(X_data, y_data, params, crossvalid = False):
#   if crossvalid:
#     num_folds = 5
#     kf = KFold(n_splits=num_folds, shuffle=True, random_state = params["seed"])
#     data_splits = [(X_data[train_ind], X_data[val_ind], y_data[train_ind], y_data[val_ind]) for train_ind, val_ind in kf.split(X_data)]
#   else:
#     num_folds = 1
#     X_train, X_val, y_train, y_val = network_training.split_data(X_data, y_data, train_size=0.8)
#     data_splits = [(X_train, X_val, y_train, y_val)]

#   for fold, (X_train_fold, X_val_fold, y_train_fold, y_val_fold) in enumerate(data_splits):
#       print(f"Fold {fold+1}/{num_folds}")

#       y_train_fold = keras.utils.to_categorical(y_train_fold, num_classes=3)
#       y_val_fold = keras.utils.to_categorical(y_val_fold, num_classes=3)

#       X_train_fold = np.expand_dims(X_train_fold, axis=-1)
#       X_val_fold = np.expand_dims(X_val_fold, axis=-1)

#       model, accuracy, f1_score, precision, recall, non_quantized_time = network_training.train_model(params, X_train_fold, y_train_fold, X_val_fold, y_val_fold)
#       tflite_quant_model = network_training.get_quantized_model(model, X_train_fold)
#       interpreter = network_training.get_tflite_interpreter(tflite_quant_model)
#       accuracy_quant, f1_score_quant, precision_quant, recall_quant, quantized_time = network_training.evaluate_quantized_metrics(interpreter, X_val_fold, y_val_fold)

#       # start_time = time.time()
#       # score = model.evaluate(X_val_fold, y_val_fold, verbose=0)
#       # non_quantized_time = time.time() - start_time

#       print(f"""Non-quantized: acc - {accuracy}, f1 - {f1_score}, prec - {precision}, rec - {recall}""")
#       print(f"""Quantized: acc - {accuracy_quant}, f1 - {f1_score_quant}, prec - {precision_quant}, rec - {quantized_time}""")
#       print(f"""Non-quantized time: {non_quantized_time}; Quantized time: {quantized_time} """)

#       if fold == 0:
#         print(model.summary())
#         non_quantized_model_size, quantized_model_size =  network_training.compare_model_sizes(tflite_quant_model, model)
#         print(f"""Non-quantized size: {non_quantized_model_size}; Quantized size: {quantized_model_size} KB """)

#       total_accuracy.append(accuracy)
#       total_f1_score.append(f1_score)
#       total_precision.append(precision)
#       total_recall.append(recall)
#       total_non_quantized_time.append(non_quantized_time)

#       total_accuracy_quant.append(accuracy_quant)
#       total_f1_score_quant.append(f1_score_quant)
#       total_precision_quant.append(precision_quant)
#       total_recall_quant.append(recall_quant)
#       total_quantized_time.append(quantized_time)

#   mean_accuracies = np.mean(total_accuracy)
#   mean_f1_scores = np.mean(total_f1_score)
#   mean_precisions = np.mean(total_precision)
#   mean_recalls = np.mean(total_recall)
#   mean_times = np.mean(total_non_quantized_time)

#   mean_accuracies_quant = np.mean(total_accuracy_quant)
#   mean_f1_scores_quant = np.mean(total_f1_score_quant)
#   mean_precisions_quant = np.mean(total_precision_quant)
#   mean_recalls_quant = np.mean(total_recall_quant)
#   mean_times_quant = np.mean(total_quantized_time)
#   print(f"""Non-quantized:  acc - {mean_accuracies*100:.2f} F1 - {mean_f1_scores*100:.2f}
#         prec - {mean_precisions*100:.2f} rec - {mean_recalls*100:.2f}
#         time - {mean_times:.2f} """)

#   print(f"""Quantized:  acc - {mean_accuracies_quant*100:.2f} F1 - {mean_f1_scores_quant*100:.2f}
#         prec - {mean_precisions_quant*100:.2f} rec - {mean_recalls_quant*100:.2f}
#         time - {mean_times_quant:.2f} """)

In [ ]:
# import keras.utils

# keras.utils.set_random_seed(params["seed"])
# np.random.seed(params["seed"])
# training_loop(X_data, y_data, params, True)